<a href="https://colab.research.google.com/github/Shaanvitha/installing-matplotlib_seaborn_numpy/blob/main/assignment_12%5BEDA%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd

# Load the dataset
df = pd.read_csv('adult_with_headers.csv')

# Display basic information about the dataset
print(df.info())

# Display summary statistics
print(df.describe())

# Check for missing values
print(df.isnull().sum())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       32561 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education       32561 non-null  object
 4   education_num   32561 non-null  int64 
 5   marital_status  32561 non-null  object
 6   occupation      32561 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital_gain    32561 non-null  int64 
 11  capital_loss    32561 non-null  int64 
 12  hours_per_week  32561 non-null  int64 
 13  native_country  32561 non-null  object
 14  income          32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB
None
                age        fnlwgt  education_num  capital_gain  capital_loss  \
count  3

In [4]:
# Option 1: Impute missing values with the mean (for numerical) or mode (for categorical)
# select numerical columns
numerical_cols = df.select_dtypes(include=['number']).columns
df[numerical_cols] = df[numerical_cols].fillna(df[numerical_cols].mean())

# Option 2: Remove rows with missing values
df.dropna(inplace=True)


In [5]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Separating numerical features
numerical_features = df.select_dtypes(include=['int64', 'float64']).columns

# Standard Scaling
scaler = StandardScaler()
df[numerical_features] = scaler.fit_transform(df[numerical_features])

# Min-Max Scaling (for comparison)
scaler = MinMaxScaler()
df[numerical_features] = scaler.fit_transform(df[numerical_features])


In [6]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

# Separating categorical features
categorical_features = df.select_dtypes(include=['object']).columns

# Apply One-Hot Encoding to categorical variables with less than 5 categories
for col in categorical_features:
    if df[col].nunique() < 5:
        df = pd.get_dummies(df, columns=[col], drop_first=True)

# Apply Label Encoding to categorical variables with more than 5 categories
for col in categorical_features:
    if col in df.columns and df[col].nunique() >= 5:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col])


In [8]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

# Separating categorical features
categorical_features = df.select_dtypes(include=['object']).columns

# Apply Label Encoding to categorical variables with more than 5 categories
for col in categorical_features:
    if col in df.columns and df[col].nunique() >= 5:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col])

In [15]:
import numpy as np

# Re-establish numerical_features if it was overwritten
numerical_features = df.select_dtypes(include=['int64', 'float64']).columns

# Identify skewed features
skewed_features = df[numerical_features].apply(lambda x: x.skew()).sort_values(ascending=False)
skewed_feature = skewed_features.index[0]  # Take the most skewed feature for demonstration

# Apply log transformation
df[skewed_feature] = np.log1p(df[skewed_feature])

In [10]:
from sklearn.ensemble import IsolationForest

# Apply Isolation Forest
iso_forest = IsolationForest(contamination=0.05)
outliers = iso_forest.fit_predict(df)

# Remove outliers
df = df[outliers != -1]


In [11]:
!pip install ppscore
import ppscore as pps
import pandas as pd

# Sample DataFrame (replace this with your actual data loading)
data = {'col1': [1, 2, 3], 'col2': [4, 5, 6]}
df = pd.DataFrame(data)

# Calculate PPS matrix
pps_matrix = pps.matrix(df)

# Compare with correlation matrix
correlation_matrix = df.corr()

print(pps_matrix)
print(correlation_matrix)

      x     y  ppscore            case  is_valid_score metric  baseline_score  \
0  col1  col1        1  predict_itself            True   None               0   
1  col1  col2        0   unknown_error           False   None               0   
2  col2  col1        0   unknown_error           False   None               0   
3  col2  col2        1  predict_itself            True   None               0   

   model_score model  
0            1  None  
1            0  None  
2            0  None  
3            1  None  
      col1  col2
col1   1.0   1.0
col2   1.0   1.0
